# Random Forest
## Introduction
Random forests are essentially a collection of decision trees that are each fit on a subsample of the data. While an individual tree is typically noisey and subject to high variance, random forests average many different trees to reduce the variability and leave us with a powerful classifier. Random forests are also non-parametric and require little to no parameter tuning. They differ from many common machine learning models used today that are typically optimized using gradient descent. Models like linear regression, support vector machines, neural networks, etc. require a lot of matrix based operations, while on the other hand tree based models like random forest are constructed with just basic arithmetic.

## Explanation of Algorithm in Python
### Import Required Libraries
Import the necessary libraries: numpy for numerical operations, pandas for data manipulation, sklearn for the diabetes dataset, and matplotlib for plotting.


In [85]:
import random
import pandas as pd
import numpy as np

### Preparing the Data
We list out the features by taking all columns except the last one (df.columns[:-1]), which is presumed to be the target variable. We determine the number of samples in our training set (90% of the total) and store it in nb_train.
Using sample, we shuffle the dataset, ensuring that rows are in random order to prevent any ordering biases.
Finally, we split our data into training and testing sets. X_train and y_train contain the features and labels for training, while X_test and y_test hold the data for testing.

In [86]:
df = pd.read_csv(r"C:\Users\arsha\OneDrive - Manipal Academy of Higher Education\Desktop\Cryptonite\Sample_Datasets\random_forest_dataset.csv")
features = df.columns[:-1].tolist()
nb_train = int(np.floor(0.9 * len(df)))
df = df.sample(frac=1, random_state=207)
X_train = df[features][:nb_train]
y_train = df.iloc[:, -1][:nb_train].values
X_test = df[features][nb_train:]
y_test = df.iloc[:, -1][nb_train:].values

### Calculating Entropy
The entropy function calculates the level of impurity or randomness in a dataset. Entropy helps to measure the uncertainty of data labels, which is essential for assessing the quality of splits in a decision tree.

In [87]:
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return - (p * np.log2(p) + (1 - p) * np.log2(1-p))

### Calculating Information Gain
Parent is the combined dataset from left_child and right_child.
We calculate the proportion of positive samples in the parent, left child, and right child nodes as p_parent, p_left, and p_right, respectively.
We compute the entropy for each of these nodes (IG_p, IG_l, and IG_r).
Finally, we calculate the information gain by subtracting the weighted entropy of the children from the parent entropy. This IG helps to find the optimal split.

In [88]:
def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)
    return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r

### Generating Bootstrap Samples
The draw_bootstrap function generates bootstrapped samples by randomly selecting data points with replacement. This sampling is central to the Random Forest approach, allowing each tree to see a different data subset.

In [89]:
def draw_bootstrap(X_train, y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
    X_bootstrap = X_train.iloc[bootstrap_indices].values
    y_bootstrap = y_train[bootstrap_indices]
    X_oob = X_train.iloc[oob_indices].values
    y_oob = y_train[oob_indices]
    return X_bootstrap, y_bootstrap, X_oob, y_oob

### Finding the Optimal Split Point
Randomly sample up to max_features features from X_bootstrap.
For each feature and potential split point within it, we create two child nodes (left_child and right_child).
Continuous values are split based on whether each value is greater or less than the split point.
We calculate information gain for each split and select the one with the highest gain, setting it as the best node.

In [90]:
def find_split_point(X_bootstrap, y_bootstrap, max_features):
    feature_ls = list()
    num_features = len(X_bootstrap[0])
    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)
    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'y_bootstrap': []}
            if type(split_point) in [int, float]:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value <= split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
            else:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value == split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
            
            split_info_gain = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                node = {'information_gain': split_info_gain, 
                        'left_child': left_child, 
                        'right_child': right_child, 
                        'split_point': split_point,
                        'feature_idx': feature_idx}
    return node

### Creating Terminal Nodes
The terminal_node function assigns a final class to a node once it reaches a stopping criterion. This final class becomes the prediction of the tree for that branch.

In [91]:
def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key=y_bootstrap.count)
    return pred

### Splitting Nodes Recursively
The split_node function is a recursive function that splits a node into left and right children based on the best split found. It stops splitting when certain conditions are met, such as reaching the maximum depth, a minimum number of samples, or when a terminal node is reached.

In [92]:
def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']    
    del(node['left_child'])
    del(node['right_child'])
    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return
    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node
    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_features, min_samples_split, max_depth, depth + 1)
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

### Building a Decision Tree
The root node is determined by finding the best split point on the entire bootstrapped dataset.
The split_node function is called to split this root node and continue branching recursively until a stopping condition is met. The completed tree is returned as root_node.

In [93]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    return root_node

### Making Predictions with a Decision Tree
We check if the value of the test sample at feature_idx is less than or equal to the split point in the tree.
Depending on the split, we either move to the left or right child node.
If a child node is a terminal node, we return the predicted value; otherwise, we continue traversing until reaching a terminal node.

In [94]:
def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']
    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']

### Out-Of-Bag (OOB) Score Calculation
The oob_score function calculates the error rate for Out-Of-Bag samples, giving an unbiased estimate of model accuracy on unseen data.

In [95]:
def oob_score(tree, X_test, y_test):
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != y_test[i]:
            mis_label += 1
    return mis_label / len(X_test)

### Building the Random Forest
The random_forest function builds an ensemble of decision trees by bootstrapping data and calculating the OOB error for each tree.

In [96]:
def random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_samples_split):
    tree_ls = list()
    oob_ls = list()
    for i in range(n_estimators):
        X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)
        tree = build_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split)
        tree_ls.append(tree)
        oob_error = oob_score(tree, X_oob, y_oob)
        oob_ls.append(oob_error)
    print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
    return tree_ls

### Making Predictions with the Random Forest
For each sample in X_test, predictions are generated by each tree in tree_ls and stored in ensemble_preds.
The majority vote is applied to the predictions to determine the final label for each sample (final_pred).
The final predictions are returned as a NumPy array.

In [97]:
def predict_rf(tree_ls, X_test):
    pred_ls = list()
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test.values[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key=ensemble_preds.count)
        pred_ls.append(final_pred)
    return np.array(pred_ls)

### Training and Evaluating the Random Forest Model
We define hyperparameters for the Random Forest: number of trees (n_estimators), maximum features per split (max_features), tree depth (max_depth), and minimum samples per split (min_samples_split).
Using random_forest, we train the model on X_train and y_train.
The trained model makes predictions on X_test using predict_rf.
Finally, we compute and print the accuracy by comparing predictions (preds) with true labels (y_test).

In [98]:
n_estimators = 100
max_features = 3
max_depth = 10
min_samples_split = 2
model = random_forest(X_train, y_train, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2)
preds = predict_rf(model, X_test)
acc = sum(preds == y_test) / len(y_test)
print("Testing accuracy: {}".format(np.round(acc,3)))

OOB estimate: 0.50
Testing accuracy: 0.56


# Theory for Random Forest
### Entropy
Entropy is a measurement of impurity (uncertainty) that uses the following formula:

$$H(X) = -\sum_{j} p_{j} \log_{2} p_{j}$$

where $p_{j}$ is the probability of class $j$. In the case of binary classification, entropy takes on the form:

$$H(X) = -p \log_{2} p - (1-p) \log_{2} (1-p)$$

where $p$ denotes $P(X=1)$ (the probability of a positive outcome). Also, note that in the case of binary classification, we use $\log_{2}$.  
Entropy follows a very intuitive interpretation with the following properties:

- Certainty: Entropy is minimized when all samples in a node belong to the same class such that $P(X = 1) = 1$ (in our case every passenger survives)
  $$
  -1 \log_2(1) - 0 \log_2(0) = 0
  $$

- Uncertainty: Entropy is maximized when we have a uniform class distribution such that $P(X = 1) = 0.5$ (in our case each passenger has a 50% chance of surviving)
$$
-0.5 \log_2(0.5) - 0.5 \log_2(0.5) = 0.5 + 0.5 = 1
$$

When we are looking for a training set to split, we'll want to find a set that maximizes entropy such that half the outcomes are positive and the other half negative, so we'll want to start uncertain. When we are looking for a value to split, we'll want to minimize entropy, so we'll want to end as certain as possible.